In [15]:
# Step 1: Import the necessary libraries
import boto3
import json
import re

# Step 2: Set up the AWS Textract client
textract = boto3.client('textract', region_name='us-east-1')

In [16]:
# Define your S3 bucket and document name
s3_bucket = "fp-prod-s3"  # Replace with your actual bucket name
s3_file = "000A7DE8-9724-4E69-8AEB-840C3A467C43.jpg"  # Replace with your actual file name in S3

In [17]:
try:
    response = textract.analyze_expense(
        Document={
            'S3Object': {
                'Bucket': s3_bucket,
                'Name': s3_file
            }
        })


except Exception as e_raise:
    print(e_raise)
    raise e_raise

In [18]:
# Debug: Print the response to check the raw data returned by Amazon Textract
print("Raw Response from Textract:")
print(json.dumps(response, indent=4))

Raw Response from Textract:
{
    "DocumentMetadata": {
        "Pages": 1
    },
    "ExpenseDocuments": [
        {
            "ExpenseIndex": 1,
            "SummaryFields": [
                {
                    "Type": {
                        "Text": "ADDRESS",
                        "Confidence": 54.277610778808594
                    },
                    "LabelDetection": {
                        "Text": "SHIP TO:",
                        "Geometry": {
                            "BoundingBox": {
                                "Width": 0.057496387511491776,
                                "Height": 0.01099281758069992,
                                "Left": 0.07712353020906448,
                                "Top": 0.25249946117401123
                            },
                            "Polygon": [
                                {
                                    "X": 0.07712353020906448,
                                    "Y": 0.25253796577453613
       

In [36]:
# # Step 4: Initialize an empty dictionary to hold our output data
# output = {
#     "card_in": None,
#     "card_out": None,
#     "truck_number": None,
#     "truck_license_no": None,
#     "trailer_no": None,
#     "driver_no": None,
#     "carrier": None,
#     "products": []
# }

output={}

In [ ]:
# # Step 5: Define common regex patterns for key-value extraction
# patterns = {
#     "card_in": r"Card In[:\s]*(\d{2}:\d{2} \d{2}/\d{2}/\d{4})",
#     "card_out": r"Card Out[:\s]*(\d{2}:\d{2} \d{2}/\d{2}/\d{4})",
#     "truck_number": r"Truck Number[:\s]*(\S+)",
#     "truck_license_no": r"Truck License No[:\s]*(\S+)",
#     "trailer_no": r"Trailer No[:\s]*(\S+)",
#     "driver_no": r"Driver No[:\s]*(\S+)",
#     "carrier": r"Carrier[:\s]*(.*)"
# }

In [37]:
# Step 6: Extract relevant data from the SummaryFields
summary_fields = response.get('ExpenseDocuments', [])[0].get('SummaryFields', [])

In [38]:
# Debug: Print SummaryFields to verify the data structure
print("\nSummary Fields:")
print(json.dumps(summary_fields, indent=4))


Summary Fields:
[
    {
        "Type": {
            "Text": "ADDRESS",
            "Confidence": 54.277610778808594
        },
        "LabelDetection": {
            "Text": "SHIP TO:",
            "Geometry": {
                "BoundingBox": {
                    "Width": 0.057496387511491776,
                    "Height": 0.01099281758069992,
                    "Left": 0.07712353020906448,
                    "Top": 0.25249946117401123
                },
                "Polygon": [
                    {
                        "X": 0.07712353020906448,
                        "Y": 0.25253796577453613
                    },
                    {
                        "X": 0.1344807893037796,
                        "Y": 0.25249946117401123
                    },
                    {
                        "X": 0.13461992144584656,
                        "Y": 0.263462632894516
                    },
                    {
                        "X": 0.07726896554231644,
    

In [39]:
# Extracting the summary fields from the response
summary_fields = response.get('ExpenseDocuments', [])[0].get('SummaryFields', [])

# Prepare an empty dictionary to store the extracted fields and values
extracted_data = {}

# Loop through each summary field
for field in summary_fields:
    # Get field type (normalized type) and value
    field_type = field.get('Type', {}).get('Text', '')
    value = field.get('ValueDetection', {}).get('Text', '')
    
    # Check if LabelDetection exists and get the label
    label = field.get('LabelDetection', {}).get('Text', '')
    
    # If LabelDetection exists, use it as the label
    if label:
        extracted_data[label] = value
    else:
        # If LabelDetection doesn't exist, try using field_type as the label
        extracted_data[field_type] = value

# Convert the extracted data to a JSON format
json_output = json.dumps(extracted_data, indent=4)

# Print the resulting JSON
print("Extracted JSON:")
print(json_output)

Extracted JSON:
{
    "SHIP TO:": "VARIOUS TX",
    "CITY": "VARIOUS",
    "ADDRESS_BLOCK": "VARIOUS",
    "CARRIER:": "CARRIER: 0007403 MOFFITT HOLDINGS DBA MOFFITT\nSERVICES",
    "NAME": "ENERGY\nTRANSFER",
    "ACCOUNT:": "8002548501 MOFFITT HOLDINGS - TX",
    "Document#": "0000223363",
    "PO Number:": "",
    "TOTALS[GALS]:": "7442",
    "VENDOR_NAME": "ENERGY\nTRANSFER",
    "EMERGENCY CONTACT:": "Chemtrec 1-800-424-9300 / 703-527-3887",
    "VENDOR_PHONE": "703-527-3887",
    "SHIPMENT OF ORIGIN:": "ET JC NOLAN TERMINAL\n-\n2883 IH-20 SOUTH SERVICE RD\nSTANTON TX 79782",
    "Folio:": "10/012",
    "3rd Party Ref:": "000007336",
    "Order:": "0000000000",
    "TCN:": "T74TX2764",
    "Load Start:": "2024-10-12 06: 09",
    "Load Stop:": "2024-10-12 06: 22",
    "SUPPLIER:": "0000000450 SUNOCO, LLC",
    "CUSTOMER:": "0000000048 Sunoco Whsl Contract Gas/Dist",
    "SCAC:": "MFVS",
    "DRIVER:": "00055135 EVERARDO RODRIGUEZ",
    "TANK(s):": "0100, 0012, 0011",
    "METER(s):

In [41]:
import json

# Extracting the summary fields from the response
summary_fields = response.get('ExpenseDocuments', [])[0].get('SummaryFields', [])

# Prepare an empty dictionary to store the extracted fields and values
extracted_data = {}

# Loop through each summary field
for field in summary_fields:
    # Get field type (normalized type) and value
    field_type = field.get('Type', {}).get('Text', '')
    value = field.get('ValueDetection', {}).get('Text', '')
    
    # Check if LabelDetection exists and get the label
    label = field.get('LabelDetection', {}).get('Text', '')
    
    # If LabelDetection exists, use it as the label
    if label:
        extracted_data[label] = value
    else:
        # If LabelDetection doesn't exist, try using field_type as the label
        extracted_data[field_type] = value


# Append the extracted data to the output dictionary
output.update(extracted_data)

# Convert the extracted data to a JSON format
json_output = json.dumps(output, indent=4)

# Print the resulting JSON
print("Extracted JSON:")
print(json_output)


Extracted JSON:
{
    "SHIP TO:": "VARIOUS TX",
    "CITY": "VARIOUS",
    "ADDRESS_BLOCK": "VARIOUS",
    "CARRIER:": "CARRIER: 0007403 MOFFITT HOLDINGS DBA MOFFITT\nSERVICES",
    "NAME": "ENERGY\nTRANSFER",
    "ACCOUNT:": "8002548501 MOFFITT HOLDINGS - TX",
    "Document#": "0000223363",
    "PO Number:": "",
    "TOTALS[GALS]:": "7442",
    "VENDOR_NAME": "ENERGY\nTRANSFER",
    "EMERGENCY CONTACT:": "Chemtrec 1-800-424-9300 / 703-527-3887",
    "VENDOR_PHONE": "703-527-3887",
    "SHIPMENT OF ORIGIN:": "ET JC NOLAN TERMINAL\n-\n2883 IH-20 SOUTH SERVICE RD\nSTANTON TX 79782",
    "Folio:": "10/012",
    "3rd Party Ref:": "000007336",
    "Order:": "0000000000",
    "TCN:": "T74TX2764",
    "Load Start:": "2024-10-12 06: 09",
    "Load Stop:": "2024-10-12 06: 22",
    "SUPPLIER:": "0000000450 SUNOCO, LLC",
    "CUSTOMER:": "0000000048 Sunoco Whsl Contract Gas/Dist",
    "SCAC:": "MFVS",
    "DRIVER:": "00055135 EVERARDO RODRIGUEZ",
    "TANK(s):": "0100, 0012, 0011",
    "METER(s):

In [42]:
output

{'SHIP TO:': 'VARIOUS TX',
 'CITY': 'VARIOUS',
 'ADDRESS_BLOCK': 'VARIOUS',
 'CARRIER:': 'CARRIER: 0007403 MOFFITT HOLDINGS DBA MOFFITT\nSERVICES',
 'NAME': 'ENERGY\nTRANSFER',
 'ACCOUNT:': '8002548501 MOFFITT HOLDINGS - TX',
 'Document#': '0000223363',
 'PO Number:': '',
 'TOTALS[GALS]:': '7442',
 'VENDOR_NAME': 'ENERGY\nTRANSFER',
 'EMERGENCY CONTACT:': 'Chemtrec 1-800-424-9300 / 703-527-3887',
 'VENDOR_PHONE': '703-527-3887',
 'SHIPMENT OF ORIGIN:': 'ET JC NOLAN TERMINAL\n-\n2883 IH-20 SOUTH SERVICE RD\nSTANTON TX 79782',
 'Folio:': '10/012',
 '3rd Party Ref:': '000007336',
 'Order:': '0000000000',
 'TCN:': 'T74TX2764',
 'Load Start:': '2024-10-12 06: 09',
 'Load Stop:': '2024-10-12 06: 22',
 'SUPPLIER:': '0000000450 SUNOCO, LLC',
 'CUSTOMER:': '0000000048 Sunoco Whsl Contract Gas/Dist',
 'SCAC:': 'MFVS',
 'DRIVER:': '00055135 EVERARDO RODRIGUEZ',
 'TANK(s):': '0100, 0012, 0011',
 'METER(s):': '030201,030251,030253,030101,030153',
 'CCN': '631608',
 'Driver/Operator:': 'EVERARDO ROD

In [31]:
# Debugging: Print the fields and values
print("Summary Fields:")
for field in summary_fields:
    field_type = field.get('Type', {}).get('Text', 'Unknown Type')
    value_detection = field.get('ValueDetection', {}).get('Text', 'No Value Detected')
    label_detection = field.get('LabelDetection', {}).get('Text', 'No Label Detected')

    # Print the extracted fields with type and value
    print(f"Field Type: {field_type}")
    print(f"Detected Text (Value): {value_detection}")
    if label_detection != 'No Label Detected':
        print(f"Label: {label_detection}")
    print("------------------------")

Summary Fields:
Field Type: ADDRESS
Detected Text (Value): VARIOUS TX
Label: SHIP TO:
------------------------
Field Type: CITY
Detected Text (Value): VARIOUS
------------------------
Field Type: ADDRESS_BLOCK
Detected Text (Value): VARIOUS
------------------------
Field Type: NAME
Detected Text (Value): CARRIER: 0007403 MOFFITT HOLDINGS DBA MOFFITT
SERVICES
Label: CARRIER:
------------------------
Field Type: NAME
Detected Text (Value): ENERGY
TRANSFER
------------------------
Field Type: ACCOUNT_NUMBER
Detected Text (Value): 8002548501 MOFFITT HOLDINGS - TX
Label: ACCOUNT:
------------------------
Field Type: INVOICE_RECEIPT_ID
Detected Text (Value): 0000223363
Label: Document#
------------------------
Field Type: PO_NUMBER
Detected Text (Value): 
Label: PO Number:
------------------------
Field Type: RECEIVER_ADDRESS
Detected Text (Value): VARIOUS TX
Label: SHIP TO:
------------------------
Field Type: RECEIVER_NAME
Detected Text (Value): CARRIER: 0007403 MOFFITT HOLDINGS DBA MOFFIT

In [40]:
# Step 8: Extract product line item details from the LineItemGroups
line_item_groups = response.get('ExpenseDocuments', [])[0].get('LineItemGroups', [])

# Debug: Print LineItemGroups to verify the data structure
print("\nLine Item Groups:")
print(json.dumps(line_item_groups, indent=4))


Line Item Groups:
[
    {
        "LineItemGroupIndex": 1,
        "LineItems": [
            {
                "LineItemExpenseFields": [
                    {
                        "Type": {
                            "Text": "ITEM",
                            "Confidence": 99.93122863769531
                        },
                        "ValueDetection": {
                            "Text": "NA1993, Fuel Oil 3, PG III, ERG#128, 1 T/T 405300",
                            "Geometry": {
                                "BoundingBox": {
                                    "Width": 0.265114426612854,
                                    "Height": 0.01055819820612669,
                                    "Left": 0.07703004777431488,
                                    "Top": 0.37219497561454773
                                },
                                "Polygon": [
                                    {
                                        "X": 0.07703004777431488,
      

In [ ]:
# # Step 9: Process LineItemGroups to extract product details
# for line_item_group in line_item_groups:
#     for line_item in line_item_group.get('LineItems', []):
#         product_details = {}
#         for field in line_item.get('LineItemExpenseFields', []):
#             field_type = field.get('Type', {}).get('Text', '').upper()
#             value = field.get('ValueDetection', {}).get('Text', '').strip()
            
#             if field_type == "ITEM":
#                 product_details["product_name"] = value
#             elif field_type == "PRICE":
#                 product_details["price"] = value
#             elif field_type == "QUANTITY":
#                 product_details["quantity"] = value
#             elif field_type == "OCT":
#                 product_details["OCT"] = value
#             elif field_type == "TEMP":
#                 product_details["temp"] = value
#             elif field_type == "GRAV/DENS":
#                 product_details["gravity"] = value
#             elif field_type == "GROSS":
#                 product_details["gross_gallons"] = value
#             elif field_type == "NET":
#                 product_details["net_gallons"] = value
        
#         # Add product details to the products list if it's complete
#         if product_details:
#             output["products"].append(product_details)

KeyError: 'products'

In [46]:
output

{'SHIP TO:': 'VARIOUS TX',
 'CITY': 'VARIOUS',
 'ADDRESS_BLOCK': 'VARIOUS',
 'CARRIER:': 'CARRIER: 0007403 MOFFITT HOLDINGS DBA MOFFITT\nSERVICES',
 'NAME': 'ENERGY\nTRANSFER',
 'ACCOUNT:': '8002548501 MOFFITT HOLDINGS - TX',
 'Document#': '0000223363',
 'PO Number:': '',
 'TOTALS[GALS]:': '7442',
 'VENDOR_NAME': 'ENERGY\nTRANSFER',
 'EMERGENCY CONTACT:': 'Chemtrec 1-800-424-9300 / 703-527-3887',
 'VENDOR_PHONE': '703-527-3887',
 'SHIPMENT OF ORIGIN:': 'ET JC NOLAN TERMINAL\n-\n2883 IH-20 SOUTH SERVICE RD\nSTANTON TX 79782',
 'Folio:': '10/012',
 '3rd Party Ref:': '000007336',
 'Order:': '0000000000',
 'TCN:': 'T74TX2764',
 'Load Start:': '2024-10-12 06: 09',
 'Load Stop:': '2024-10-12 06: 22',
 'SUPPLIER:': '0000000450 SUNOCO, LLC',
 'CUSTOMER:': '0000000048 Sunoco Whsl Contract Gas/Dist',
 'SCAC:': 'MFVS',
 'DRIVER:': '00055135 EVERARDO RODRIGUEZ',
 'TANK(s):': '0100, 0012, 0011',
 'METER(s):': '030201,030251,030253,030101,030153',
 'CCN': '631608',
 'Driver/Operator:': 'EVERARDO ROD

In [47]:
type(output)

dict

In [48]:
# Append product details to the "products" list in the output dictionary
output["products"] = []  # Initialize the products list

# Loop through each line item group and extract product details
for line_item_group in line_item_groups:
    for line_item in line_item_group.get("LineItems", []):
        product_details = {}
        
        for field in line_item.get('LineItemExpenseFields', []):
            field_type = field.get('Type', {}).get('Text', '').upper()
            value = field.get('ValueDetection', {}).get('Text', '').strip()
            
            # Map the field types to the product details
            if field_type == "ITEM":
                product_details["product_name"] = value
            elif field_type == "PRICE":
                product_details["price"] = value
            elif field_type == "QUANTITY":
                product_details["quantity"] = value
            elif field_type == "OCT":
                product_details["OCT"] = value
            elif field_type == "TEMP":
                product_details["temp"] = value
            elif field_type == "GRAV/DENS":
                product_details["gravity"] = value
            elif field_type == "GROSS":
                product_details["gross_gallons"] = value
            elif field_type == "NET":
                product_details["net_gallons"] = value
        
        # If product details are complete, add them to the "products" list
        if product_details:
            output["products"].append(product_details)

# Convert the extracted data to a JSON format
json_output = json.dumps(output, indent=4)

# Print the resulting JSON
print("Extracted JSON:")
print(json_output)

Extracted JSON:
{
    "SHIP TO:": "VARIOUS TX",
    "CITY": "VARIOUS",
    "ADDRESS_BLOCK": "VARIOUS",
    "CARRIER:": "CARRIER: 0007403 MOFFITT HOLDINGS DBA MOFFITT\nSERVICES",
    "NAME": "ENERGY\nTRANSFER",
    "ACCOUNT:": "8002548501 MOFFITT HOLDINGS - TX",
    "Document#": "0000223363",
    "PO Number:": "",
    "TOTALS[GALS]:": "7442",
    "VENDOR_NAME": "ENERGY\nTRANSFER",
    "EMERGENCY CONTACT:": "Chemtrec 1-800-424-9300 / 703-527-3887",
    "VENDOR_PHONE": "703-527-3887",
    "SHIPMENT OF ORIGIN:": "ET JC NOLAN TERMINAL\n-\n2883 IH-20 SOUTH SERVICE RD\nSTANTON TX 79782",
    "Folio:": "10/012",
    "3rd Party Ref:": "000007336",
    "Order:": "0000000000",
    "TCN:": "T74TX2764",
    "Load Start:": "2024-10-12 06: 09",
    "Load Stop:": "2024-10-12 06: 22",
    "SUPPLIER:": "0000000450 SUNOCO, LLC",
    "CUSTOMER:": "0000000048 Sunoco Whsl Contract Gas/Dist",
    "SCAC:": "MFVS",
    "DRIVER:": "00055135 EVERARDO RODRIGUEZ",
    "TANK(s):": "0100, 0012, 0011",
    "METER(s):

In [5]:
# import boto3

# textract = boto3.client('textract',region_name='us-east-1')

# with open('../test_bol.jpg', 'rb') as document:
#     response = textract.analyze_document(
#         Document={'Bytes': document.read()},
#         FeatureTypes=['FORMS', 'TABLES']
#     )

# # Process response
# for block in response['Blocks']:
#     if block['BlockType'] == 'KEY_VALUE_SET':
#         print(f"Key: {block['Key']['Text']}, Value: {block['Value']['Text']}")

In [1]:

import boto3
import json
import logging
from logging.handlers import RotatingFileHandler

# Step 1: Set up logging configuration to log to a file
log_file = "textract_processing.log"

# Set up a rotating log handler to limit the size of the log file and keep old logs
log_handler = RotatingFileHandler(log_file, maxBytes=5 * 1024 * 1024, backupCount=3)  # 5 MB max, 3 backups
log_handler.setLevel(logging.INFO)

# Create a formatter with detailed log messages
log_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
log_handler.setFormatter(log_formatter)

# Add the handler to the logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(log_handler)

# Step 2: Set up the AWS Textract client
textract = boto3.client('textract', region_name='us-east-1')

# Step 3: Define your S3 bucket and document name
s3_bucket = "fp-prod-s3"  # Your S3 bucket name
s3_file = "040644B1-FFA1-4665-A8C0-B9EFCB3814B7.jpg"  # Replace with your actual file name in S3

def extract_text_blocks(response):
    """
    Extracts blocks of text from the Textract response and returns them as a list of text lines.
    """
    extracted_data = []
    blocks = response.get('Blocks', [])

    for block in blocks:
        block_type = block.get('BlockType', '')
        if block_type == 'LINE':
            text = block.get('Text', '').strip()
            if text:
                extracted_data.append(text)

    return extracted_data


def extract_tables(response):
    """
    Extracts table data from the Textract response.
    """
    tables = []
    blocks = response.get('Blocks', [])

    # Iterate over blocks to extract tables
    for block in blocks:
        if block.get('BlockType') == 'TABLE':
            table = []
            for relationship in block.get('Relationships', []):
                if relationship['Type'] == 'CHILD':
                    for child_id in relationship['Ids']:
                        child_block = next(b for b in blocks if b['Id'] == child_id)
                        if child_block.get('BlockType') == 'CELL':
                            row_index = child_block['RowIndex']
                            column_index = child_block['ColumnIndex']
                            text = child_block.get('Text', '').strip()

                            # Add text data to corresponding table cell (row x column)
                            while len(table) <= row_index:
                                table.append([])
                            table[row_index - 1].append(text)

            tables.append(table)
    return tables


def extract_forms(response):
    """
    Extracts form data from the Textract response.
    """
    forms = []
    blocks = response.get('Blocks', [])

    for block in blocks:
        if block.get('BlockType') == 'KEY_VALUE_SET':
            key = ''
            value = ''
            for item in block.get('EntityTypes', []):
                if item == 'KEY':
                    key = block.get('Text', '').strip()
                elif item == 'VALUE':
                    value = block.get('Text', '').strip()

            if key and value:
                forms.append({'key': key, 'value': value})

    return forms


def main():
    try:
        # Call the Textract AnalyzeDocument API with the input document in S3
        logger.info(f"Processing file {s3_file} from bucket {s3_bucket}...")
        
        response = textract.analyze_document(
            Document={
                'S3Object': {
                    'Bucket': s3_bucket,
                    'Name': s3_file
                }
            },
            FeatureTypes=['TABLES', 'FORMS']  # Extract tables and forms
        )

        # Initialize the output dictionary
        output = {}

        # Extract text blocks
        logger.info("Extracting text blocks...")
        text_data = extract_text_blocks(response)
        output["text"] = text_data

        # Extract tables
        logger.info("Extracting tables...")
        table_data = extract_tables(response)
        output["tables"] = table_data

        # Extract form data
        logger.info("Extracting forms...")
        form_data = extract_forms(response)
        output["forms"] = form_data

        # Convert the extracted data to a JSON format
        json_output = json.dumps(output, indent=4)

        # Print the resulting JSON
        logger.info("Extraction complete. Here is the extracted data:")
        logger.info(json_output)

        # Optional: print the result to stdout
        print(json_output)

    except Exception as e:
        logger.error(f"An error occurred: {e}")
        raise e


if __name__ == "__main__":
    main()


{
    "text": [
        "COMPANY:",
        "TSO LGSTCS OPERATIONS LLC",
        "PAGE 1 of 1",
        "LOADING TICKET / BILL OF LADING",
        "ORIGIN:",
        "150 SOLANO WAY MARTINEZ, CA, MARTINEZ, CA",
        "BOL NUMBER:",
        "518062188",
        "REV: 0 (028840)",
        "TRIP #1823",
        "FOLIO #: 20250506",
        "START LOAD:",
        "05/20/2025 01:44:16",
        "END LOAD: 05/20/2025 01:54:32",
        "COMPANY REGISTRATION #: 6396",
        "SOLD TO / CUSTOMER / CONSIGNEE:",
        "SHIP TO / DESTINATION:",
        "40013079 TRUMAN ARNOLD COMPANIES",
        "40013079 TRUMAN ARNOLD COMPANIES",
        "VARIOUS, CA",
        "VARIOUS",
        "CA USA",
        "CUSTOMER ACCOUNT #: 89000",
        "LOADING NUMBER NE9000",
        "SUPPLIER NESTE TERMINALING",
        "P.O. #",
        "NESTE US INC",
        "REQUISITION:",
        "DEST: 9000",
        "RECEIVED BY / CARRIER: 5320 PACIFIC STATES PETROLEUM INC",
        "SCAC CODE PSPI",
        "TRAILER 